In [1]:
import paho.mqtt.client as paho
import numpy as np
import random
import json
import networkx as nx
from node import EpidemicGraph, TransitionDiagram, calc_R0
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from datetime import datetime
from IPython.display import clear_output
# %matplotlib inline

In [2]:
t = [' ' for i in range(50)]
flush = ''.join(t)
# broker_ip = 'localhost'
broker_ip = '10.101.21.2'
connflag = False

In [3]:
def plot(G, title='step_1', ns=1000, fs=12, lim=2, figsize=(5, 5)):
    pos = nx.get_node_attributes(G, 'position')
    ids = {i:i for i in range(N)}
    print('Total node number', N)

    s_a = [x for x,y in G.nodes(data=True) if y['state']=='S_a']
    s_s = [x for x,y in G.nodes(data=True) if y['state']=='S_s']
    i1_a = [x for x,y in G.nodes(data=True) if y['state']=='I1_a']
    i1_s = [x for x,y in G.nodes(data=True) if y['state']=='I1_s']
    i2_a = [x for x,y in G.nodes(data=True) if y['state']=='I2_a']
    i2_s = [x for x,y in G.nodes(data=True) if y['state']=='I2_s']


    fig = plt.figure(figsize=figsize)
    nx.draw_networkx_edges(G, pos, alpha=0.8)
    nx.draw_networkx_nodes(G, pos, node_size=ns, nodelist=s_a, node_color='#66ff99')
    nx.draw_networkx_nodes(G, pos, node_size=ns, nodelist=s_s, node_color='#006600')
    nx.draw_networkx_nodes(G, pos, node_size=ns, nodelist=i1_a, node_color='#ff6666')
    nx.draw_networkx_nodes(G, pos, node_size=ns, nodelist=i1_s, node_color='#cc0000')
    nx.draw_networkx_nodes(G, pos, node_size=ns, nodelist=i2_a, node_color='#3399ff')
    nx.draw_networkx_nodes(G, pos, node_size=ns, nodelist=i2_s, node_color='#0000cc')
    nx.draw_networkx_labels(G, pos, ids, font_size=fs)

    plt.title(title)
    red_patch = mpatches.Patch(color='red', label='I1')
    blue_patch = mpatches.Patch(color='blue', label='I2')
    green_patch = mpatches.Patch(color='green', label='S')
    plt.legend(handles=[green_patch, red_patch, blue_patch])
    plt.xlim(0, lim)
    plt.ylim(0, lim)
    return plt, fig

In [ ]:
mqttc = paho.Client()
# mqttc.max_inflight_messages_set(0)
counter = 0
N = 10
format_date = "%d %b %Y %H%M%S.%f"

def on_connect(client, userdata, flags, rc):
    global connflag
    #if connection is successful, rc value will be 0
    print("Connection returned result: " + str(rc) )
    connflag = True

def on_message(client, userdata, msg): 
    print("-t {} | -p {}".format(msg.topic, msg.payload.decode()) )
    
def on_state(client, userdata, msg): 
    global G, counter
    counter += 1
    name = datetime.now().strftime(format_date)
    js = json.loads(msg.payload.decode())
    G.node[js['pi_id']]['state'] = js['state']
#     if counter%N == 0:
#         plt, fig = plot(G, 'step {}'.format((counter-1)//N), **plt_set)
#         fig.savefig('figures/{}.png'.format(name))
#         plt.close()
    log = "-t {} | -p {}".format(msg.topic, msg.payload.decode())
    print( log)
    with open("log.txt", "a") as myfile:
        myfile.write("{},{}\n".format(name, log))
        
def on_finish(client, userdata, msg):  # on finish 
    global node_set
    log = "-t {} | -p {}".format(msg.topic, msg.payload.decode())
    with open("log_node_set.txt", "a") as myfile:
        myfile.write("{}\n".format(log))
    node_id = msg.payload.decode()
    node_set.discard(node_id)
    print(node_set, flush, end='\r')
#     with open("log_node_set.txt", "a") as myfile:
#         myfile.write("{},{}\n".format(node_set))
    if len(node_set) == 0:
#         mqttc.publish('next', 'dummy_msg', qos=2)
        node_set = {str(x) for x in range(N)}
#         clear_output(wait=True)

def on_finish_trans(client, userdata, msg):  # on finish step
    global node_set
    log = "-t {} | -p {}".format(msg.topic, msg.payload.decode())
    with open("log_node_set.txt", "a") as myfile:
        myfile.write("{}\n".format(log))
    node_id = msg.payload.decode()
    node_set.discard(node_id)
    print(node_set, flush, end='\r')
#     with open("log_node_set.txt", "a") as myfile:
#         myfile.write("{},{}\n".format(node_set))
    if len(node_set) == 0:
#         mqttc.publish('next', 'dummy_msg', qos=2)
        node_set = {str(x) for x in range(N)}
        clear_output(wait=True)

    

mqttc.on_connect = on_connect
mqttc.on_message = on_message
mqttc.message_callback_add("state", on_state)
mqttc.message_callback_add("finish", on_finish)
mqttc.message_callback_add("finish_trans", on_finish_trans)

mqttc.connect(broker_ip)
mqttc.loop_start()
mqttc.subscribe('state', 1)
mqttc.subscribe('finish', 1)
mqttc.subscribe('finish_trans', 1)

In [ ]:
# 0.04 0.05|0.04 0.04|0.3 0.3  |0.05 0.2 |0.04 0.1   R1<1 R2<1    
# 0.04 0.15|0.04 0.04|0.25 0.07|0.06 0.12|0.05 0.15  R1<1 R2>1
# 0.25 0.05|0.04 0.04|0.25 0.5 |0.05 0.12|0.04 0.15  R1>1 R2<1
# 0.25 0.35|0.04 0.04|0.04 0.35|0.06 0.07|0.05 0.06  R1>1 R2>1
paramet = {    # R1<1 R2<1  
        'alpha': [0.04, 0.05], # infect rate 
        'mu': [0.04, 0.04], # sleep s 
        'gamma': [0.3, 0.3], # rec rate 
        'lambda': [0.05, 0.2], # sleep I1 
        'kappa': [0.04, 0.1] # sleep I2    % [I2_s -> I2_a, I2_a -> I2_s]
    }
paramet = {    # R1<1 R2<1  
        'alpha': [0.04, 0.15], # infect rate 
        'mu': [0.04, 0.04], # sleep s 
        'gamma': [0.25, 0.07], # rec rate 
        'lambda': [0.06, 0.12], # sleep I1 
        'kappa': [0.04, 0.15] # sleep I2    % [I2_s -> I2_a, I2_a -> I2_s]
    }
# paramet = {    # R1<1 R2<1  
#         'alpha': [0.04, 0.05], # infect rate 
#         'mu': [0.04, 0.04], # sleep s 
#         'gamma': [0.3, 0.3], # rec rate 
#         'lambda': [0.05, 0.2], # sleep I1 
#         'kappa': [0.04, 0.1] # sleep I2    % [I2_s -> I2_a, I2_a -> I2_s]
#     }
# paramet = {    # R1<1 R2<1  
#         'alpha': [0.04, 0.05], # infect rate 
#         'mu': [0.04, 0.04], # sleep s 
#         'gamma': [0.3, 0.3], # rec rate 
#         'lambda': [0.05, 0.2], # sleep I1 
#         'kappa': [0.04, 0.1] # sleep I2    % [I2_s -> I2_a, I2_a -> I2_s]
#     }
paramet = {
        'alpha': [0.1, 0.35], # infect rate 
        'mu': [0.02, 0.02], # sleep s 
        'gamma': [0.05, 0.02], # rec rate 
        'lambda': [0.03, 0.04], # sleep I1 
        'kappa': [0.04, 0.05] # sleep I2    % [I2_s -> I2_a, I2_a -> I2_s]
    }
np.random.seed(315)  # structure
random.seed(6)       # infected
N = 50*2
r = 1
number_subgraphs = 0
node_set = {str(x) for x in range(N)}

while number_subgraphs != 1:
    a = EpidemicGraph(N,r,paramet,I1_a=10, I2_a=10, grid_size=[6, 6], create=True) # net='Uniform Grid'
    a.load_csv('100 nodes.csv',10,10)
    number_subgraphs = len(list(nx.connected_component_subgraphs(a.G)))
    number_isolated_nodes = len(list(nx.isolates(a.G)))
    print(calc_R0(a.G, paramet), (number_subgraphs, number_isolated_nodes))

G = a.G
# G.node[0]['state'] = 'S_s'
print(calc_R0(a.G, paramet))
plt_set = {
    'ns': 200, 
    'fs': 10,
    'lim': 10.1, 
    'figsize': (5,5)
}
plt, fig = plot(G, **plt_set) #  ns=300, fs=6,lim=5
plt.show()
fig.savefig('figures/_.png')
# with open("log.txt", "w") as myfile:
#     myfile.write("")

In [ ]:
G.node[1]
node_set = {str(x) for x in range(N)}
msg_list = []
for node_id in range(N):
    neighbours = list(G[node_id])
    msg = {
        "id": node_id,
        "state": G.node[node_id]['state'],
        "neighbours": neighbours
    }
    msg_list.append(msg)
    print(msg)
    
mqttc.publish('init', json.dumps(msg_list))
mqttc.publish('paramet', json.dumps(paramet))

In [ ]:
# node_set = {x: str(x) for x in range(5)}
# # mqttc.publish('finish', '14', qos=2)
# unwanted = ['1','2','3','4','0'] # set()
# {e for e in node_set if e not in unwanted}
# random.sample(list(node_set.values()), len(node_set.values()))

In [ ]:
# mqttc.loop_stop()

In [ ]:
# mqttc.publish('nextstep', 'dummy_msg', qos=2)

In [ ]:
# mqttc.publish('stop', 'dummy_msg', qos=2)

In [ ]:
# mqttc.publish('kill', 'process die', qos=2)

In [ ]:
# mqttc.publish('paramet', json.dumps(paramet), qos=2)
# mqttc.publish('finish', str(92), qos=2)

In [ ]:
mqttc.publish('start', 'dummy_msg', qos=2)